It's time to add Memory

BUT WAIT!

We have this Graph maintaining the state and appending to the state

Why isn't this handling memory?

This is a crucial point for understanding LangGraph

A super-step can be considered a single iteration over the graph nodes. Nodes that run in parallel are part of th same super-step

One "Super-Step" of the graph represents one invocation of passing messages between agents

In idiomatic langGraph, you call invoke to run your graph for each super-step, for each iteration

The reducer handles state updates automatically within one super-step, but not between them

That is what checkpointing achieves

In [3]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver() # This is a checkpoint that saves the state in memory

In [5]:
from langgraph.graph import StateGraph, START, END
from typing import Annotated
from pydantic import BaseModel
from langgraph.graph import add_messages

class State(BaseModel):
    messages: Annotated[list, add_messages]


In [6]:
tool_search = SerperDevTool()

NameError: name 'SerperDevTool' is not defined

In [ ]:
tools = [tool_search]

In [7]:
from langchain_openai import ChatOpenAI
graph_builder = StateGraph(State)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile(checkpoint=memory)
display(Image(graph.get_graph().draw_mermaid_png()))



NameError: name 'tools' is not defined

In [ ]:
import gradio as gr
config = {"configurable": {"thread_id": 1}}
# thread_id: This is the thread id for the graph. It is used to identify the thread in the graph, (it is conversational thread)

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

d:\Jeevan\Agentic_AI\LangGraph_Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
